### TODO:
- [ ] latency summary에 추가하기
- [ ] frame drop baseline performance 비교하는 figure 만들기; 기존 (using summary)
- [ ] frame drop baseline들 backlog 증가 그래프 그리는 figure 만들기; 기존 (using data)
- [ ] offloading baseline performance 비교하는 figure 만들기; fraction & accuracy & latency 비교; simulation에서 쓰던 형태 (using data & summary)
- [ ] offloading baseline들 backlog 증가 그래프 그리는 figure 만들기; 기존 (using data)
- [ ] multi agent 성능 비교 하는 figure 만들기
- [ ] multi agent에서 각 source node의 backlog 증가 그래프 그려서 비교하기

### Import libraries

In [7]:
import os, sys, time, re, random
import pandas as pd 
import numpy as np  
import matplotlib.pyplot as plt 
import seaborn as sns
import csv, json, ast  

In [ ]:
### User define libraries
import visualization as vis
import data_manage as dm
import lrlo as lrlo
import warnings

warnings.filterwarnings("ignore")

### Define methods

In [8]:

def extract_value_from_latency(value_str):
    return float(value_str.split('ms')[0].strip())

def get_latency_exp(csv_path):
    if os.path.exists(csv_path):
        df = pd.read_csv(csv_path)
        df['latency'] = df['latency'].apply(extract_value_from_latency)
        return df
        
def get_backlog_exp(csv_path):
    if os.path.exists(csv_path):
        df = pd.read_csv(csv_path)
        return df[['sum', 'avg']]
    
def calculate_mean_latency(df):
    mean_value = df['latency'].mean()
    return mean_value


### Define setting

In [20]:
experiment_path = 'data/experiment'
frame_drop_summary_path = 'data/summary/frame_dropping.csv'
computation_offloading_summary_path = 'data/summary/computation_offloading.csv'
multi_agent_summary_path = 'data/summary/multi_agent.csv'

dataset_list = ['JN', 'JK', 'SD']
exp_type_list = ['frame_dropping', 'computation_offloading', 'multi_agent']
frame_drop_method_list = ['LRLO', 'Reducto', 'FrameHopper', 'CAO']
computation_offloading_method_list = ['LRLO','JDPCRA', 'TLDOC']

#### 0. get average latency value

In [21]:
MS_TO_S = 1000

for dataset in dataset_list:
    for exp_type in exp_type_list:
        if exp_type == 'frame_dropping':
            for method in frame_drop_method_list:
                base_dir = f'{experiment_path}/{exp_type}/{method}_{dataset}'
                
                if method != 'CAO':
                    subfolder_paths = [os.path.join(base_dir, name) for name in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, name))]
                    for subfolder_path in subfolder_paths:
                        latency_file_path = f'{subfolder_path}/latency/test job 1.csv'
                        df = get_latency_exp(latency_file_path)
                        mean_latency = calculate_mean_latency(df) / MS_TO_S
                        print(f'{dataset}\t{exp_type}\t{method}\tmean latency: {mean_latency}')
                
                else:
                    latency_file_path = f'{base_dir}/latency/test job 1.csv'
                    df = get_latency_exp(latency_file_path)
                    mean_latency = calculate_mean_latency(df) / MS_TO_S
                
                    print(f'{dataset}\t{exp_type}\t{method}\tmean latency: {mean_latency}')
        
        elif exp_type == 'computation_offloading':
            for method in computation_offloading_method_list:
                base_dir = f'{experiment_path}/{exp_type}/{method}_{dataset}'
                
                if method == 'LRLO':
                    subfolder_paths = [os.path.join(base_dir, name) for name in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, name))]
                    for subfolder_path in subfolder_paths:
                        latency_file_path = f'{subfolder_path}/latency/test job 1.csv'
                        df = get_latency_exp(latency_file_path)
                        mean_latency = calculate_mean_latency(df) / MS_TO_S
                        print(f'{dataset}\t{exp_type}\t{method}\tmean latency: {mean_latency}')
                
                else:
                    latency_file_path = f'{base_dir}/latency/test job 1.csv'
                    df = get_latency_exp(latency_file_path)
                    mean_latency = calculate_mean_latency(df) / MS_TO_S
                
                    print(f'{dataset}\t{exp_type}\t{method}\tmean latency: {mean_latency}')
        

JN	frame_dropping	LRLO	mean latency: 1.6094955538603697
JN	frame_dropping	LRLO	mean latency: 1.7579586878728815
JN	frame_dropping	LRLO	mean latency: 26.951011964480607
JN	frame_dropping	LRLO	mean latency: 24.657114469021245
JN	frame_dropping	LRLO	mean latency: 2.1621528393556484
JN	frame_dropping	LRLO	mean latency: 40.54465721888366
JN	frame_dropping	LRLO	mean latency: 34.93247690288793
JN	frame_dropping	LRLO	mean latency: 2.191208302552463
JN	frame_dropping	LRLO	mean latency: 1.7173717986659665
JN	frame_dropping	Reducto	mean latency: 30.317076163746297
JN	frame_dropping	Reducto	mean latency: 3.775056211800532
JN	frame_dropping	FrameHopper	mean latency: 28.386903660019968
JN	frame_dropping	FrameHopper	mean latency: 5.052483357297665
JN	frame_dropping	CAO	mean latency: 24.81877510508135
JN	computation_offloading	LRLO	mean latency: 1.6094955538603697
JN	computation_offloading	LRLO	mean latency: 1.7579586878728815
JN	computation_offloading	LRLO	mean latency: 26.951011964480607
JN	computat

In [ ]:
exp_df = pd.read_csv('data/csvs/experiment/collective_result.csv', skipinitialspace=True)
exp_df = dm.revise_df(exp_df, 'divide', 'latency', 1000)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

datas = ['JK', 'SD', 'JN']
exp_df_stack = exp_df.melt(id_vars=['method', 'data'], value_vars=['fraction', 'accuracy', 'latency'], var_name='Metric', value_name='Value')

dm.df_info(exp_df_stack, 10)
sns.set_palette("muted")
for i, data in enumerate(datas):
    plt.figure()
    df = exp_df_stack[exp_df_stack['data'] == data]

    ax = sns.barplot(x='method', y='Value', hue='Metric', data=df[df['Metric'] != 'latency'], ci=None, width=0.5, edgecolor='0', palette="viridis")
    ax.set_xlabel('')
    plt.ylabel('value (%)', fontsize=15) 
    
    # Metric이 latency인 경우에만 오른쪽 축에 선 그래프를 그립니다.
    ax.tick_params(axis='x', rotation=45, labelsize=12)  # x축 레이블 크기를 조정합니다.
    
    df_latency = df[df['Metric'] == 'latency']
    if not df_latency.empty:
        ax2 = ax.twinx()
        sns.lineplot(x='method', y='Value', data=df_latency, marker='o', color="red", ax=ax2, linewidth=2)
        ax2.set_ylabel('latency (s)')

    ax2.set_xlabel('')
    plt.ylabel('Average latency (s)', fontsize=15) 
    #ax.set_xlabel('method')  # x축 라벨명칭 수정
    ax2.tick_params(axis='x', rotation=45, labelsize=12)  # x축 레이블 크기를 조정합니다.
    
    ax.legend().remove()
    #if data == 'JK':
    #    ax.legend(('fraction', 'accuracy'), loc='lower center', bbox_to_anchor=(.5, 1.15), ncol=2, frameon=False, fontsize=14)
    plt.tick_params(labelsize=12) 
    plt.tight_layout()  # 그래프 간 간격을 조정하여 겹침을 방지합니다
    plt.savefig(SAVE_ROOT + "exp_performance_" + data + ".svg", bbox_inches='tight', pad_inches=0)
    plt.show()

In [ ]:
def calculate_mean_latency_exp(df, length):
    mean_value = df['latency'].sum() / len(df)
    if length > len(df) + 5:    
        sum_value = df['latency'].sum() 
        sum_value += (length-len(df)-5)*300000
        mean_value = sum_value / length
        #new_row = pd.DataFrame({'latency': [300000] * (length-len(df)-5)})
        #df = pd.concat([df, new_row ], ignore_index=True) 
        #dm.df_info(df)
    return mean_value, df

In [ ]:
# 전체 실험결과가 다 뽑고싶은 당신을 위해
skip = False
if not skip:
    mean_latency_list = []
    latency_df = {'JK':{}, 'SD':{}, 'JN':{}}
    backlog_df = {'JK':{}, 'SD':{}, 'JN':{}}

    data_dir_list = ['data/csvs/experiment/results_baseline/', 'data/csvs/experiment/results_LRLO/']
    dataset = ['JK', 'SD', 'JN']
            
    for data_dir in data_dir_list:
        for data in dataset:
            folder_path = os.path.join(data_dir, data)
            for root, dirs, files in os.walk(folder_path):
                for directory in dirs:
                    if directory == 'latency':
                        name = re.split(r"[/\\]",root)[-1]
                        
                        # parts = name.split("_")   
                        # prefix = parts[0] + "_" + name.split("_")[3]
                        if name.split("_")[0] == 'LRLO':
                            prefix = name.split("_")[0] + "_" + name.split("_")[3]
                        else:
                            if name.split("_")[0] == 'frameHopper':
                                prefix = "FrameHopper" + "_" + name.split("_")[2]
                            elif name.split("_")[0] == 'reducto':
                                prefix = "Reducto" + "_" + name.split("_")[2]
                        csv_path = os.path.join(root, directory, 'test job 1.csv')
                        #if prefix in latency_df[data]:
                        #    continue
                        latency_df[data][prefix]= get_latency_exp(csv_path)
                        path_path = os.path.join(root, 'path', 'path.csv')
                        df = pd.read_csv(path_path)
                        job_length = len(df)
                        mean_latency, latency_df[data][prefix] = calculate_mean_latency_exp(latency_df[data][prefix], job_length)
                        mean_latency_list.append([data+"_"+prefix, str(mean_latency)])
                    elif directory == 'backlog':
                        name = re.split(r"[/\\]",root)[-1]
                        if name.split("_")[0] == 'LRLO':
                            prefix = name.split("_")[0] + "_" + name.split("_")[3]
                        else:
                            if name.split("_")[0] == 'frameHopper':
                                prefix = "FrameHopper" + "_" + name.split("_")[2]
                            elif name.split("_")[0] == 'reducto':
                                prefix = "Reducto" + "_" + name.split("_")[2]
                        csv_path = os.path.join(root, directory, 'total_backlog.csv')
                        #if prefix in backlog_df[data]:
                        #    print("in")
                        #    continue
                        backlog_df[data][prefix] = get_backlog_exp(csv_path)

                    #print(prefix)
        
    with open("data/csvs/experiment/mean_latency.csv", "w",  newline='') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerows(mean_latency_list)

In [ ]:
# key_list = ['frameHopper_0.7', 'frameHopper_0.9', 'reducto_0.7', 'reducto_0.9', 'LRLO_1000']
#lrlo_list = ['LRLO_1000', 'LRLO_10000', 'LRLO_100000', 'LRLO_1000000', 'LRLO_10000000', 'LRLO_100000000']
#for lrlo in lrlo_list:
#    print(lrlo)
key_list = ['FrameHopper_0.7', 'Reducto_0.7', "LRLO_1000000"]
columns = ['avg', 'sum']
target_data = 'JK'
sns.set_palette("muted")
for data in dataset:
    if data != target_data:
        continue
    data_df = backlog_df[data]
    for column in columns:
        #print(column)
        flag = True
        for key, df in data_df.items():
            #print(data_df.keys())
            if key_list[-1] not in data_df.keys():
                flag = False
                break
            #print(key)
            if key not in key_list:
                continue
            # x_values = np.arange(0, len(df) * 0.1, 0.1)
            df.reindex(np.arange(0, len(df) * 0.1, 0.1))
            sampled_df = dm.get_sampled_df(df, 10)
            #x_values = np.arange(0, len(sampled_df) * 0.1, 0.1)
            #sns.lineplot(x=latency_masking_df.index, y=latency_masking_df[column], alpha=0.5)
            if key[0] == 'L':
                key = "LRLO"
            sns.lineplot(x=sampled_df.index, y=sampled_df[column], label=key, lw=1.5, palette="Muted")
            #sns.lineplot(x=latency_unmasking_df.index, y=latency_unmasking_df[column], alpha=0.5)
        if flag:
            print("dataset", data)
            plt.xlim(0, 2800)
            plt.ylim(0, 300000000)
            plt.xlabel('Time (s)', fontsize=16)  
            plt.ylabel('Backlog (B)', fontsize=16) 
            plt.tick_params(labelsize=12) 
            plt.legend(loc='upper left', fontsize=12) 
            plt.savefig(SAVE_ROOT+"exp_backlog_"+column+".pdf") 
            idx += 1
            plt.show()
